In [13]:
import torch
import torch.nn as nn

In [14]:
from SimpleEEnvironment import BitFlipEnvironment

In [15]:
env = BitFlipEnvironment(n_bits=4)

In [16]:
import pybullet_envs
import gym
import numpy as np
from gym import wrappers
from soft_actor_critic import SoftActorCritic

In [17]:
env = gym.make('InvertedPendulumBulletEnv-v0')
env.action_space, env.observation_space.shape

(Box(-1.0, 1.0, (1,), float32), (5,))

In [18]:
sac = SoftActorCritic(env.observation_space.shape[0], action_dim=env.action_space.shape[0])

In [19]:
best_score = env.reward_range[0]
score_history = []
load_checkpoint = False

In [20]:
n_episodes = 100
max_size = 10_000
batch_size = 2
learning_freq = 32

In [21]:
from replay_buffer import ReplayBuffer

replay_buffer = ReplayBuffer(max_size)

In [22]:
def play_and_record(env, agent, buffer):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action = sac.choose_action(state)
        next_state, reward, done, _ = env.step(action)

        score += reward
        buffer.add(state, action, reward, next_state, done)

        state = next_state
    
    return score

In [23]:
n_episodes = 10_000

In [24]:
from tqdm import tqdm

for i in tqdm(range(n_episodes)):
    score = play_and_record(env, sac, replay_buffer)
    batch = replay_buffer.sample(batch_size)
    sac.learning_step(batch)
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    if avg_score > best_score:
        best_score = avg_score
        sac.save_checkpoint()
    if i % 500 == 0 and i > 0:
        print(f"iteration {i} score {score:.3f} avg_score {avg_score:.3f}")
    

  0%|          | 0/10000 [00:00<?, ?it/s]

argv[0]=
argv[0]=


 10%|█         | 1008/10000 [00:22<02:29, 60.28it/s]

iteration 1000 score 10.000 avg_score 10.930


 17%|█▋        | 1682/10000 [00:42<03:30, 39.52it/s]


KeyboardInterrupt: 